## Rugby Munster ETL

### Extracting Data from XML Files

In [38]:
import xml.etree.ElementTree as ET
import pandas as pd

def parse_xml(file):
    """This function parses the XML file and extracts the MatchData, TeamData, and FixData, including the FXID."""
    
    tree = ET.parse(file)
    root = tree.getroot()

    # Extract FXID
    fxid = root.find('.//FXID').text if root.find('.//FXID') is not None else None

    # MatchData
    match_data = [action.attrib for action in root.findall('.//MatchData/ActionRow')]
    match_data_df = pd.DataFrame(match_data)
    if fxid:
        match_data_df['FXID'] = fxid

    # TeamData
    team_data = [team.attrib for team in root.findall('.//TeamData/Player')]
    team_data_df = pd.DataFrame(team_data)
    if fxid:
        team_data_df['FXID'] = fxid

    # FixData
    fix_data = [fixture.attrib for fixture in root.findall('.//FixData/Data')]
    fix_data_df = pd.DataFrame(fix_data)
    if fxid:
        fix_data_df['FXID'] = fxid

    return match_data_df, team_data_df, fix_data_df

# Empty DataFrames
match_data_concat = pd.DataFrame()
team_data_concat = pd.DataFrame()
fix_data_concat = pd.DataFrame()

# XML Paths
xml_files = ['Data\938734_EDINvMUNS_advanced_superscout.xml', 'Data\938749_MUNSvOSPR_advanced_superscout.xml', 'Data\938754_MUNSvGLAS_advanced_superscout.xml']  # Añadir los nombres de tus archivos

# Iterate over each XML file and concatenate the data
for file in xml_files:
    match_data, team_data, fix_data = parse_xml(file)
    match_data_concat = pd.concat([match_data_concat, match_data], ignore_index=True)
    team_data_concat = pd.concat([team_data_concat, team_data], ignore_index=True)
    fix_data_concat = pd.concat([fix_data_concat, fix_data], ignore_index=True)


## Data Manipulation

In [39]:
def determine_home_away(row, fix_data_concat):
    # Filtering the FixData DataFrame for the FXID
    match = fix_data_concat[fix_data_concat['FXID'] == row['FXID']]
    
    if not match.empty:
        if row['Club'] == match.iloc[0]['FxHTID']:
            return 'Home'
        elif row['Club'] == match.iloc[0]['FxATID']:
            return 'Away'
    return 'Unknown'

# Apply the function to the TeamData DataFrame
team_data_concat['HomeAway'] = team_data_concat.apply(determine_home_away, axis=1, fix_data_concat=fix_data_concat)


,PLID,ShirtNo,Club,PosID,PLFORN,PLSURN,TEAMNAME,MINS,AttMinutes,DefMinutes,BallInPlayMins,FXID,HomeAway
133,38593,19,1750,5,Max,Williamson,Glasgow Warriors,23,6,5,11,938754,Away
134,38594,20,1750,7,Euan,Ferrie,Glasgow Warriors,11,3,2,5,938754,Away
135,21982,21,1750,6,Henco,Venter,Glasgow Warriors,23,6,5,11,938754,Away
136,35689,22,1750,14,Jamie,Dobie,Glasgow Warriors,23,6,5,11,938754,Away
137,29901,23,1750,16,Ross,Thompson,Glasgow Warriors,0,0,0,0,938754,Away


In [40]:
# Save the concatenated DataFrames as CSV to import them into Power BI
match_data_concat.to_csv('Output/match_data.csv', index=False)
team_data_concat.to_csv('Output/team_data.csv', index=False)
fix_data_concat.to_csv('Output/fix_data.csv', index=False)

print("The data has been extracted and saved as CSV files.")

The data has been extracted and saved as CSV files.
